In [ ]:
!pip install -q keras-bert
!wget -q https://storage.googleapis.com/bert_models/2018_11_03/chinese_L-12_H-768_A-12.zip
!unzip -o chinese_L-12_H-768_A-12.zip

Archive:  chinese_L-12_H-768_A-12.zip
  inflating: chinese_L-12_H-768_A-12/bert_model.ckpt.meta  
  inflating: chinese_L-12_H-768_A-12/bert_model.ckpt.data-00000-of-00001  
  inflating: chinese_L-12_H-768_A-12/vocab.txt  
  inflating: chinese_L-12_H-768_A-12/bert_model.ckpt.index  
  inflating: chinese_L-12_H-768_A-12/bert_config.json  


In [ ]:
SEQ_LEN = 512
BATCH_SIZE = 32
EPOCHS = 5

In [ ]:
import os
pretrained_path = 'chinese_L-12_H-768_A-12'
config_path = os.path.join(pretrained_path, 'bert_config.json')
checkpoint_path = os.path.join(pretrained_path, 'bert_model.ckpt')
vocab_path = os.path.join(pretrained_path, 'vocab.txt')


# 讀取seg

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
import json
import numpy  as np
import pandas as pd
from glob import glob 
from tqdm import tqdm

In [ ]:
df = pd.read_pickle('../content/drive/My Drive/Nomura/data.pickle')

In [ ]:
y = pd.read_csv('../content/drive/My Drive/Nomura/nomura_redemp.csv')

In [ ]:
df.head()

,time,author,type,title,score,filename,content,seg
0,2018-02-01 22:50:42,ted5566,新聞,台灣央行新掌門人如何走出自己的路？,10,pttdata\M.1517496645.A.38D.json,\n1.原文連結：\n\nhttps://cn.reuters.com/article/fo...,"[\n, 1, ., 原文, 連結, ：, \n, \n, \n, \n, \n, 2, ...."
1,2018-02-02 00:12:28,j853540,標的,大亞 空,3,pttdata\M.1517501551.A.D55.json,\n\n\n\n1. 標的：大亞\n\n2. 分類：空\n\n3. 分析/正文：\n\n最近...,"[\n, \n, \n, \n, 1, ., , 標的, ：, 大亞\n, \n, 2, ..."
2,2018-02-02 08:11:27,milk7054,新聞,蘋果本季營收預測不如預期 iPhone整體需,19,pttdata\M.1517530291.A.04E.json,\n-------------------------------發文提醒---------...,"[\n, -------------------------------, 發文, 提醒, ..."
3,2018-02-02 08:25:45,kaiming7955,新聞,蘋果營收、盈餘創新高，稱愛瘋X週週奪最,14,pttdata\M.1517531147.A.7F3.json,\n\n蘋果周四於美股盤後公佈前季（10-12月）財報，雖然愛瘋出貨量不如預期，但仍舊是營收...,"[\n, \n, 蘋果, 周四, 於, 美股, 盤, 後, 公佈, 前, 季, （, 10,..."
4,2018-02-02 08:30:04,ESunBoy,閒聊,2018/02/02 盤中閒聊,569,pttdata\M.1517531406.A.3CD.json,\nxx老師早料到台股準備封關，\n2018全面毀滅，\n你一半我一半，\n大獲全勝。\n\...,"[\n, xx, 老師, 早, 料到, 台股, 準備, 封關, ，, \n, 2018, 全..."


In [ ]:
y.head()

,ym,redemption,delta,signal
0,2014-11-30,0.037555,NaN,False
1,2014-12-31,0.044058,0.006504,True
2,2015-01-31,0.038173,-0.005886,False
3,2015-02-28,0.038173,0.000000,False
4,2015-03-31,0.123520,0.085347,True


# 加入stopwords字典

In [ ]:
word_sentence_list = df['seg']

In [ ]:
stopWords=[]
with open('../content/drive/My Drive/Nomura/stopword.txt', 'r', encoding='UTF-8') as file:
    for data in file.readlines():
        data = data.strip()
        stopWords.append(data)
        remainder_sentence = []

stopWords.append('\n')
apndix = ['(', '61', '.231', '.', '200', '.', '192', ' ', '臺灣', ')', ' ', '05', '/', '01', '/', '2020', ' ', '15', ':', '34', ':', '14\n',  '--\n', '※ '  ]
for i in apndix:
    stopWords.append(i)


In [ ]:
for i in word_sentence_list:
    remainderWords = list(filter(lambda a: a not in stopWords and a != '\n', i))
    remainder_sentence.append(remainderWords)


In [ ]:
print(remainder_sentence[0:2])

[['原文', '連結', '原文', '內容', '台湾', '央行', '正式', '进入', '后彭时', '代', '新掌门', '走出', '路', '路透', '台北', '2月', '1日', ' - ', '20', '年', '前', '台灣', '桃園', '大園', '場', '嚴重', '空難', '台灣', '央行', '人才', '嚴重', '折損', '意外', '造就', '彭淮南', '楊金龍', '兩', '位', '央行', '基層', '一路', '做到', '總裁', '鮮少', '知道', '當年', '擔任', '央行', '行務', '委員', '楊金龍', '原本', '出席', '印尼', '巴里島', '東南亞', '國家', '中央', '銀行', '總裁', '聯合會', 'SEACEN', '名單', '中', '時任', '央行', '總裁', '許遠東', '筆', '揮', '臨時', '改由', '出席', '事', '一度', '楊金龍', '相當', '沮喪', '知情', '人士', '稱', '當年', '台灣', '剛', '歷經', '1997年', '亞洲', '金融', '風暴', '1998年', '初', '許遠東', '慰勞', '官員', '辛勞\n', '特', '出席', 'SEACEN', '年會', '名義', '安排', '位', '一級', '主委', '一起', '赴', '巴里島', '回程', '時', '遭遇', '重大', '空難', '突然', '空難', '央行', '頓時', '群龍無首', '當時', '總統', '李登輝', '隨即', '任命', '當時', '擔任', '中國', '商銀', '現今', '兆豐', '商銀', '董事長', '彭淮南', '接掌', '總裁', '職務', '彭淮南', '提早', '數', '月', '當上', '央行', '總裁', '彭淮南\n', '迄', '依舊', '李登輝', '心', '存', '感激', '央行', '內部', '活動', '央行', '合唱團', '特別', '獻', '首', '感恩', '心', '彭淮南', '接掌', '央行', '深受', '彭信賴', '楊金龍

In [ ]:
stoppedContend=[]
for sentence in remainder_sentence:
  stoppedContend.append(''.join(sentence))

In [ ]:
print(stoppedContend[0:2])

['原文連結原文內容台湾央行正式进入后彭时代新掌门走出路路透台北2月1日 - 20年前台灣桃園大園場嚴重空難台灣央行人才嚴重折損意外造就彭淮南楊金龍兩位央行基層一路做到總裁鮮少知道當年擔任央行行務委員楊金龍原本出席印尼巴里島東南亞國家中央銀行總裁聯合會SEACEN名單中時任央行總裁許遠東筆揮臨時改由出席事一度楊金龍相當沮喪知情人士稱當年台灣剛歷經1997年亞洲金融風暴1998年初許遠東慰勞官員辛勞\n特出席SEACEN年會名義安排位一級主委一起赴巴里島回程時遭遇重大空難突然空難央行頓時群龍無首當時總統李登輝隨即任命當時擔任中國商銀現今兆豐商銀董事長彭淮南接掌總裁職務彭淮南提早數月當上央行總裁彭淮南\n迄依舊李登輝心存感激央行內部活動央行合唱團特別獻首感恩心彭淮南接掌央行深受彭信賴楊金龍官運亨通次順利出線擔任央\n行總裁職彭淮南總統蔡英文強力推薦面對前連續兩央行總裁任期過世向來篤信一貫道彭淮南接掌總裁隨即找來高人央行修風水希望永保平安如今走進綠意盎然鳥言花香央行舉凡一草一木處處風水痕跡央行舊大樓正前方總共種八不同品樹種位居中間榕樹個頭價值不斐知情人士稱榕樹代表央行主事兩旁小葉\n欖仁樹木象徵央行主要局處種沖煞央行修石獅父子座水池規定停放央\n行停車場車子車頭必須一致外央行方便內部往來新舊大樓間興建座風雨走廊興建後果真是風雨\n不斷特別走廊直通舊大樓二樓外匯局首當其衝迫使央行不得不變相關閉風水走廊期阻擋新大樓直接摜穿到外匯局煞氣彭淮南大修央行風水央行平平安安度20年頭同樣彭淮南強勢領導台灣金融業央行雙率變穩定然過於穩定市場波動市場形容台灣金融市場灘死水詬病**彭下楊楊未來挑戰**\n不到月楊金龍即將走馬上任台灣將來迎來新總裁出生台灣南端小鎮--屏東恆春楊金龍小時候家境寬裕籌措學費楊金龍\n母親暑假時群雞鴨鵝負責雞鴨鵝養大暑假結束時家\n禽養大正好賣錢繳學費賣更多錢楊金龍必須設法雞鴨鵝養更肥\n更迄楊金龍天飛走家禽特別研究年媒體閒\n話家常時三句鳥事觸及央行業務楊金龍顯得格局謹慎小心說年央行沒有聲音發言人深受彭淮南信任喜愛楊金龍完全沒有聲音擔任業務局長時媒體記者互動時不少磨\n擦向來直來直往楊金龍數度記者氣哭最後彭淮南出面緩頰而今楊金龍順利接掌央行總裁外界均稱楊金龍彭淮南一手提拔上來人選彭淮南轉任顧問會從旁協助料初期楊金龍跟隨彭淮南腳步彭規楊隨外界人事案安全評論分析師民意

# 整理資料

In [ ]:
df['stopped_content']= stoppedContend

In [ ]:
df.head()

,time,author,type,title,score,filename,content,seg,stopped_content
0,2018-02-01 22:50:42,ted5566,新聞,台灣央行新掌門人如何走出自己的路？,10,pttdata\M.1517496645.A.38D.json,\n1.原文連結：\n\nhttps://cn.reuters.com/article/fo...,"[\n, 1, ., 原文, 連結, ：, \n, \n, \n, \n, \n, 2, ....",原文連結原文內容台湾央行正式进入后彭时代新掌门走出路路透台北2月1日 - 20年前台灣桃園大...
1,2018-02-02 00:12:28,j853540,標的,大亞 空,3,pttdata\M.1517501551.A.D55.json,\n\n\n\n1. 標的：大亞\n\n2. 分類：空\n\n3. 分析/正文：\n\n最近...,"[\n, \n, \n, \n, 1, ., , 標的, ：, 大亞\n, \n, 2, ...",標的大亞\n分類空\n分析正文最近發現隻一直跌今天開盤小漲尾盤賣壓高繼續跌\n繼續空\n進退...
2,2018-02-02 08:11:27,milk7054,新聞,蘋果本季營收預測不如預期 iPhone整體需,19,pttdata\M.1517530291.A.04E.json,\n-------------------------------發文提醒---------...,"[\n, -------------------------------, 發文, 提醒, ...",-------------------------------發文提醒-----------...
3,2018-02-02 08:25:45,kaiming7955,新聞,蘋果營收、盈餘創新高，稱愛瘋X週週奪最,14,pttdata\M.1517531147.A.7F3.json,\n\n蘋果周四於美股盤後公佈前季（10-12月）財報，雖然愛瘋出貨量不如預期，但仍舊是營收...,"[\n, \n, 蘋果, 周四, 於, 美股, 盤, 後, 公佈, 前, 季, （, 10,...",蘋果周四美股盤公佈前季1012月財報愛瘋出貨量預期營收盈餘破記錄主要貢獻蘋果當季賣出730萬...
4,2018-02-02 08:30:04,ESunBoy,閒聊,2018/02/02 盤中閒聊,569,pttdata\M.1517531406.A.3CD.json,\nxx老師早料到台股準備封關，\n2018全面毀滅，\n你一半我一半，\n大獲全勝。\n\...,"[\n, xx, 老師, 早, 料到, 台股, 準備, 封關, ，, \n, 2018, 全...",xx老師早料到台股準備封關2018全面毀滅一半一半獲全勝


In [ ]:
from pandas.tseries.offsets import MonthEnd
df['time'] = pd.to_datetime(df['time'])
df['EndOfMonth'] = pd.to_datetime(df['time'], format="%Y%m") + MonthEnd(1)

In [ ]:
df.head()

,time,author,type,title,score,filename,content,seg,stopped_content,EndOfMonth
0,2018-02-01 22:50:42,ted5566,新聞,台灣央行新掌門人如何走出自己的路？,10,pttdata\M.1517496645.A.38D.json,\n1.原文連結：\n\nhttps://cn.reuters.com/article/fo...,"[\n, 1, ., 原文, 連結, ：, \n, \n, \n, \n, \n, 2, ....",原文連結原文內容台湾央行正式进入后彭时代新掌门走出路路透台北2月1日 - 20年前台灣桃園大...,2018-02-28 22:50:42
1,2018-02-02 00:12:28,j853540,標的,大亞 空,3,pttdata\M.1517501551.A.D55.json,\n\n\n\n1. 標的：大亞\n\n2. 分類：空\n\n3. 分析/正文：\n\n最近...,"[\n, \n, \n, \n, 1, ., , 標的, ：, 大亞\n, \n, 2, ...",標的大亞\n分類空\n分析正文最近發現隻一直跌今天開盤小漲尾盤賣壓高繼續跌\n繼續空\n進退...,2018-02-28 00:12:28
2,2018-02-02 08:11:27,milk7054,新聞,蘋果本季營收預測不如預期 iPhone整體需,19,pttdata\M.1517530291.A.04E.json,\n-------------------------------發文提醒---------...,"[\n, -------------------------------, 發文, 提醒, ...",-------------------------------發文提醒-----------...,2018-02-28 08:11:27
3,2018-02-02 08:25:45,kaiming7955,新聞,蘋果營收、盈餘創新高，稱愛瘋X週週奪最,14,pttdata\M.1517531147.A.7F3.json,\n\n蘋果周四於美股盤後公佈前季（10-12月）財報，雖然愛瘋出貨量不如預期，但仍舊是營收...,"[\n, \n, 蘋果, 周四, 於, 美股, 盤, 後, 公佈, 前, 季, （, 10,...",蘋果周四美股盤公佈前季1012月財報愛瘋出貨量預期營收盈餘破記錄主要貢獻蘋果當季賣出730萬...,2018-02-28 08:25:45
4,2018-02-02 08:30:04,ESunBoy,閒聊,2018/02/02 盤中閒聊,569,pttdata\M.1517531406.A.3CD.json,\nxx老師早料到台股準備封關，\n2018全面毀滅，\n你一半我一半，\n大獲全勝。\n\...,"[\n, xx, 老師, 早, 料到, 台股, 準備, 封關, ，, \n, 2018, 全...",xx老師早料到台股準備封關2018全面毀滅一半一半獲全勝,2018-02-28 08:30:04


# 資料合併

In [ ]:
df['EndOfMonth'] = df['EndOfMonth'].apply(lambda x:x.date())

In [ ]:
y.head()

,ym,redemption,delta,signal
0,2014-11-30,0.037555,NaN,False
1,2014-12-31,0.044058,0.006504,True
2,2015-01-31,0.038173,-0.005886,False
3,2015-02-28,0.038173,0.000000,False
4,2015-03-31,0.123520,0.085347,True


In [ ]:
y['ym'] = pd.to_datetime(y['ym']).apply(lambda x:x.date())

In [ ]:
data = df.merge(y,   left_on='EndOfMonth',right_on='ym')

In [ ]:
from keras_bert import load_trained_model_from_checkpoint

# BERT 透過 Subword 的方式讓辭典長度最大就是 30522
token_dict = {}
with open(vocab_path, 'r', encoding='utf8') as f:
    for line in f.readlines():
        token = line.strip()
        token_dict[token] = len(token_dict)
print("辭典長度:", len(token_dict))


model = load_trained_model_from_checkpoint(
    config_path,
    checkpoint_path,
    training=False,
    trainable=False,
    seq_len=SEQ_LEN,
)

辭典長度: 21128


In [ ]:
from keras_bert import Tokenizer
tokenizer = Tokenizer(token_dict)

# 整理成可跑型態

In [ ]:
data.head()

,time,author,type,title,score,filename,content,seg,stopped_content,EndOfMonth,ym,redemption,delta,signal
0,2018-02-01 22:50:42,ted5566,新聞,台灣央行新掌門人如何走出自己的路？,10,pttdata\M.1517496645.A.38D.json,\n1.原文連結：\n\nhttps://cn.reuters.com/article/fo...,"[\n, 1, ., 原文, 連結, ：, \n, \n, \n, \n, \n, 2, ....",原文連結原文內容台湾央行正式进入后彭时代新掌门走出路路透台北2月1日 - 20年前台灣桃園大...,2018-02-28,2018-02-28,0.120004,0.0,False
1,2018-02-02 00:12:28,j853540,標的,大亞 空,3,pttdata\M.1517501551.A.D55.json,\n\n\n\n1. 標的：大亞\n\n2. 分類：空\n\n3. 分析/正文：\n\n最近...,"[\n, \n, \n, \n, 1, ., , 標的, ：, 大亞\n, \n, 2, ...",標的大亞\n分類空\n分析正文最近發現隻一直跌今天開盤小漲尾盤賣壓高繼續跌\n繼續空\n進退...,2018-02-28,2018-02-28,0.120004,0.0,False
2,2018-02-02 08:11:27,milk7054,新聞,蘋果本季營收預測不如預期 iPhone整體需,19,pttdata\M.1517530291.A.04E.json,\n-------------------------------發文提醒---------...,"[\n, -------------------------------, 發文, 提醒, ...",-------------------------------發文提醒-----------...,2018-02-28,2018-02-28,0.120004,0.0,False
3,2018-02-02 08:25:45,kaiming7955,新聞,蘋果營收、盈餘創新高，稱愛瘋X週週奪最,14,pttdata\M.1517531147.A.7F3.json,\n\n蘋果周四於美股盤後公佈前季（10-12月）財報，雖然愛瘋出貨量不如預期，但仍舊是營收...,"[\n, \n, 蘋果, 周四, 於, 美股, 盤, 後, 公佈, 前, 季, （, 10,...",蘋果周四美股盤公佈前季1012月財報愛瘋出貨量預期營收盈餘破記錄主要貢獻蘋果當季賣出730萬...,2018-02-28,2018-02-28,0.120004,0.0,False
4,2018-02-02 08:30:04,ESunBoy,閒聊,2018/02/02 盤中閒聊,569,pttdata\M.1517531406.A.3CD.json,\nxx老師早料到台股準備封關，\n2018全面毀滅，\n你一半我一半，\n大獲全勝。\n\...,"[\n, xx, 老師, 早, 料到, 台股, 準備, 封關, ，, \n, 2018, 全...",xx老師早料到台股準備封關2018全面毀滅一半一半獲全勝,2018-02-28,2018-02-28,0.120004,0.0,False


In [ ]:
index_array = data['type']=="新聞"

In [ ]:
# index_array = data['type'].apply(lambda x: x not in (['新聞', '標的'])) 
# X=data[index_array ]['content'].iloc[0:100].apply(lambda x:tokenizer.encode(x, max_len=SEQ_LEN))
traindata = data[index_array]
indices=[]
y=[]
for content, label in zip(traindata['stopped_content'],traindata['signal'] ) :
  ids, segments = tokenizer.encode(content, max_len=SEQ_LEN)
  indices.append(ids)

  y.append(label)

# Y=data[index_array ]['label'].iloc[0:100]

In [ ]:
X = [indices, np.zeros_like(indices)]
# X

In [ ]:
import numpy as np
Xdata = [a, np.zeros_like(a)]

In [ ]:
# en, seg = tokenizer.encode(a[0])
# print("[Encoding]:", en)
# print("[Segement]:", seg)
# de = tokenizer.decode(a[0])
# print("[Decode]:", de)

In [ ]:
# data['stopped_content'][0]

In [ ]:
# # 準備資料流
# import glob
# from tqdm import tqdm
# import numpy as np

# def load_data(path):
#     global tokenizer
#     indices, sentiments = [], []
#     for folder, sentiment in (('neg', 0), ('pos', 1)):
#         pat = glob.glob("{}/{}/*".format(path, folder))
#         for fn in tqdm(iter(pat), total=len(pat)):
#             with open(fn, 'r', encoding="utf-8") as f:
#                   text = f.read()
#             ids, segments = tokenizer.encode(text, max_len=SEQ_LEN)
#             indices.append(ids)
#             sentiments.append(sentiment)
#     items = list(zip(indices, sentiments))
#     np.random.shuffle(items)
#     indices = np.array([i for i, s in items])
#     sentiments = np.array([s for i, s in items])

#     return [indices, np.zeros_like(indices)], np.array(sentiments)

# # 讀取資料集
# train_path = os.path.join(os.path.dirname(dataset), 'aclImdb', 'train')
# test_path = os.path.join(os.path.dirname(dataset), 'aclImdb', 'test')

# train_x, train_y = load_data(train_path)
# test_x, test_y = load_data(test_path)

In [ ]:
x

<tf.Tensor 'simple_rnn_1/strided_slice_2:0' shape=(None, 64) dtype=float32>

# 跑模型

In [ ]:
import keras
from keras.layers import SimpleRNN
inputs = model.inputs[:2]
x = SimpleRNN(64)(model.output)
outputs = keras.layers.Dense(units=2, activation='softmax')(x)
model = keras.models.Model(inputs, outputs)
model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer="adam",
    metrics=['sparse_categorical_accuracy']
)

ValueError: ignored

In [ ]:
model.summary()

Model: "model_5"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Input-Token (InputLayer)        (None, 512)          0                                            
__________________________________________________________________________________________________
Input-Segment (InputLayer)      (None, 512)          0                                            
__________________________________________________________________________________________________
Embedding-Token (TokenEmbedding [(None, 512, 768), ( 16226304    Input-Token[0][0]                
__________________________________________________________________________________________________
Embedding-Segment (Embedding)   (None, 512, 768)     1536        Input-Segment[0][0]              
____________________________________________________________________________________________

In [ ]:
model.fit(
    X,
    y,
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    validation_split=0.2
)

Train on 14306 samples, validate on 3577 samples
Epoch 1/5
14306/14306 [==============================] - 1735s 121ms/step - loss: 0.6964 - sparse_categorical_accuracy: 0.5139 - val_loss: 0.7155 - val_sparse_categorical_accuracy: 0.4129
Epoch 2/5
14306/14306 [==============================] - 1733s 121ms/step - loss: 0.6900 - sparse_categorical_accuracy: 0.5321 - val_loss: 0.6451 - val_sparse_categorical_accuracy: 0.7137
Epoch 3/5
14306/14306 [==============================] - 1735s 121ms/step - loss: 0.6828 - sparse_categorical_accuracy: 0.5527 - val_loss: 0.6693 - val_sparse_categorical_accuracy: 0.6153
Epoch 4/5
  384/14306 [..............................] - ETA: 23:21 - loss: 0.6730 - sparse_categorical_accuracy: 0.5990

KeyboardInterrupt: ignored

In [ ]:
# model.evaluate(test_x, test_y)